# Analyze Complaint Survey Conditions

This notebook analyzes whether BrightSpring has received more conditions in complaint surveys (referred to as **Complaint** surveys in QCOR) than other providers in the seven states with the most for-profit facilities. The analysis looks only at **smaller** (4-8 bed) facilities.

In [1]:
import pandas as pd
import glob
import datetime

In [2]:
ownership_data = (
    pd
    .concat(
        [
            pd
            .read_csv(
                f, 
                parse_dates=["termination_date", "particip_date"]
            ) for f in glob.glob("../../data/ownership/final/*.csv")
        ]
    )
)

In [3]:
ownership_data.head()

,provider_id,name,type,region,state,address,phone,address.1,particip_date,certified_beds,hospital_based,ownership_type,termination_code,termination_date,legal_owner,is_brightspring
0,51G002,5TH AVENUE GROUP HOME,Intermediate Care Facilities for Individuals w...,(III) Philadelphia,WV,"916 FIFTH AVENUE\nSAINT ALBANS, WV 25177",304 720-0015,"916 FIFTH AVENUE\nSAINT ALBANS, WV 25177",1981-10-06,9,No,For Profit,NaN,NaT,"RSCR WEST VIRGINIA, INCORPORATED",True
1,51G003,POTOMAC CENTER,Intermediate Care Facilities for Individuals w...,(III) Philadelphia,WV,"ONE BLUE STREET\nROMNEY, WV 26757",304 822-3861,"ONE BLUE STREET\nROMNEY, WV 26757",1983-11-09,24,No,Non-Profit,NaN,NaT,POTOMAC CENTER,False
2,51G007,CROSS LANES GROUP HOME,Intermediate Care Facilities for Individuals w...,(III) Philadelphia,WV,"5202 LINDA VISTA DRIVE\nCROSS LANES, WV 25313",304 776-2005,"5202 LINDA VISTA DRIVE\nCROSS LANES, WV 25313",1986-11-17,8,No,For Profit,NaN,NaT,"RSCR WEST VIRGINIA, INCORPORATED",True
3,51G008,EIGHTH AVENUE GROUP HOME,Intermediate Care Facilities for Individuals w...,(III) Philadelphia,WV,"1519 8TH AVENUE\nHUNTINGTON, WV 25701",304 523-0177,"1519 8TH AVENUE\nHUNTINGTON, WV 25701",1987-03-06,8,No,For Profit,NaN,NaT,"VOCA CORPORATION OF WEST VIRGINIA, INCORPORATED",True
4,51G009,VIRGINIA AVENUE GROUP HOME,Intermediate Care Facilities for Individuals w...,(III) Philadelphia,WV,"821 VIRGINIA AVENUE\nHUNTINGTON, WV 25701",304 523-0196,"821 VIRGINIA AVENUE\nHUNTINGTON, WV 25701",1987-03-06,8,No,For Profit,NaN,NaT,"VOCA CORPORATION OF WEST VIRGINIA, INCORPORATED",True


In [4]:
# Filter facilities by bed size
sm_state_owners = ownership_data.loc[
    lambda x: (x["certified_beds"] >= 4) &
    (x["certified_beds"] <= 8)
].copy()

In [5]:
len(sm_state_owners)

3646

In [6]:
surveys = pd.read_csv("../../data/qcor/surveys.csv", parse_dates=["date"])
deficiencies = pd.read_csv("../../data/qcor/deficiencies.csv", parse_dates=["date"])

In [7]:
surveys.head()

,survey_key,provider_id,date,type,subtype,num_deficiencies
0,01G005-2010-0,01G005,2010-11-24,STANDARD,LIFE SAFETY,43
1,01G005-2010-1,01G005,2010-11-24,STANDARD,HEALTH,5
2,01G005-2011-0,01G005,2011-07-27,STANDARD,LIFE SAFETY,76
3,01G005-2011-1,01G005,2011-07-27,STANDARD,HEALTH,18
4,01G006-2010-0,01G006,2010-07-27,STANDARD,LIFE SAFETY,0


In [8]:
survey_df = surveys.loc[
    lambda x: 
        x["provider_id"].isin(sm_state_owners["provider_id"]) & 
        (x["date"] >= "2019-03-05") & # Date KKR bought BrightSpring
        (x["date"] <= "2021-12-31") & # Last date for data
        (x["type"] == "COMPLAINT") & # Only Complaint surveys
        (x["subtype"] == "HEALTH") # Only Health surveys
].copy()

In [9]:
len(survey_df)

7464

In [10]:
deficiencies.sample(5)

,survey_key,level,tag,desc,date
318169,39G041-2018-0,Standard,K0258,Number of Exits - Patient Sleeping and Non-Sl,2018-08-28
312561,37G133-2018-1,Standard,W0323,PHYSICIAN SERVICES,2018-10-17
317614,39G036-2013-0,Standard,K0341,Fire Alarm System - Installation,2013-02-06
294533,36G402-2012-1,Standard,W0209,INDIVIDUAL PROGRAM PLAN,2012-05-31
282327,36G026-2011-1,Standard,K0352,Sprinkler System - Supervisory Signals,2011-05-02


In [11]:
con_df = deficiencies.loc[
    lambda x: 
        (x["level"] == "Condition") &
        x["survey_key"].isin(survey_df["survey_key"])
]

In [12]:
len(con_df)

558

In [13]:
con_surveys = (
    con_df
    .groupby("survey_key")["tag"]
    .count()
    .to_frame()
    .rename(columns={"tag": "condition_count"})
    .reset_index()
)

In [14]:
con_surveys.head()

,survey_key,condition_count
0,05G080-2019-0,1
1,05G104-2020-2,1
2,05G148-2020-0,2
3,05G149-2019-1,2
4,05G149-2021-2,2


In [15]:
last_survey_date = survey_df["date"].max()

In [16]:
last_survey_date

Timestamp('2021-12-31 00:00:00')

In [17]:
acquisition_date = pd.Timestamp(year=2019, month=3, day=5)
end_date = pd.Timestamp(year=2021, month=12, day=31)

In [18]:
def calc_overall_days_open(row):
    if pd.isnull(row["termination_date"]): # facility still open
        if row["particip_date"] <= acquisition_date: # open the whole time
            delta = end_date - acquisition_date
            return delta.days
        else: # opened during time, still open
            delta = last_survey_date - row["particip_date"]
            return delta.days
    else:
        if row["particip_date"] <= acquisition_date: # opened before time, now closed
            delta = row["termination_date"] - acquisition_date
            return delta.days
        else: # opened after start, now closed
            delta = row["termination_date"] - row["particip_date"]
            return delta.days

In [19]:
sm_state_owners.dtypes

provider_id                 object
name                        object
type                        object
region                      object
state                       object
address                     object
phone                       object
address.1                   object
particip_date       datetime64[ns]
certified_beds               int64
hospital_based              object
ownership_type              object
termination_code            object
termination_date    datetime64[ns]
legal_owner                 object
is_brightspring               bool
dtype: object

In [20]:
sm_state_owners["days_open"] = (
    sm_state_owners
    .apply(
        lambda x: calc_overall_days_open(x), axis=1
    )
)

sm_state_owners["bed_days_open"] = (
    sm_state_owners
    .apply(
        lambda x: x["days_open"] * x["certified_beds"], axis=1
    )
)

In [21]:
sm_state_owners.head()

,provider_id,name,type,region,state,address,phone,address.1,particip_date,certified_beds,hospital_based,ownership_type,termination_code,termination_date,legal_owner,is_brightspring,days_open,bed_days_open
2,51G007,CROSS LANES GROUP HOME,Intermediate Care Facilities for Individuals w...,(III) Philadelphia,WV,"5202 LINDA VISTA DRIVE\nCROSS LANES, WV 25313",304 776-2005,"5202 LINDA VISTA DRIVE\nCROSS LANES, WV 25313",1986-11-17,8,No,For Profit,NaN,NaT,"RSCR WEST VIRGINIA, INCORPORATED",True,1032,8256
3,51G008,EIGHTH AVENUE GROUP HOME,Intermediate Care Facilities for Individuals w...,(III) Philadelphia,WV,"1519 8TH AVENUE\nHUNTINGTON, WV 25701",304 523-0177,"1519 8TH AVENUE\nHUNTINGTON, WV 25701",1987-03-06,8,No,For Profit,NaN,NaT,"VOCA CORPORATION OF WEST VIRGINIA, INCORPORATED",True,1032,8256
4,51G009,VIRGINIA AVENUE GROUP HOME,Intermediate Care Facilities for Individuals w...,(III) Philadelphia,WV,"821 VIRGINIA AVENUE\nHUNTINGTON, WV 25701",304 523-0196,"821 VIRGINIA AVENUE\nHUNTINGTON, WV 25701",1987-03-06,8,No,For Profit,NaN,NaT,"VOCA CORPORATION OF WEST VIRGINIA, INCORPORATED",True,1032,8256
5,51G010,811 S. KANAWHA GROUP HOME,Intermediate Care Facilities for Individuals w...,(III) Philadelphia,WV,"811 S. KANAWHA STREET\nBECKLEY, WV 25801",304 252-5937,"811 S. KANAWHA STREET\nBECKLEY, WV 25801",1987-02-17,8,No,For Profit,NaN,NaT,"VOCA CORPORATION OF WEST VIRGINIA, INCORPORATED",True,1032,8256
6,51G011,1204 S. KANAWHA GROUP HOME,Intermediate Care Facilities for Individuals w...,(III) Philadelphia,WV,"1204 S. KANAWHA STREET\nBECKLEY, WV 25801",304 252-5676,"1204 S. KANAWHA STREET\nBECKLEY, WV 25801",1987-02-17,8,No,For Profit,NaN,NaT,"VOCA CORPORATION OF WEST VIRGINIA, INCORPORATED",True,1032,8256


## Analysis Versus Everyone

In [22]:
overall_results = (
    survey_df
    .merge(
        con_surveys,
        how="left",
        on="survey_key"
    )
    .assign(condition_count = lambda df: df["condition_count"].fillna(0))
    .merge(
        sm_state_owners[[
            "provider_id", 
            "name", 
            "state", 
            "is_brightspring", 
            "bed_days_open"
        ]]
    )
    .groupby(["state", "is_brightspring"])
    .pipe(lambda grp: pd.DataFrame({
        "facs": grp["provider_id"].nunique(),
        "surveys": grp["survey_key"].nunique(),
        "conditions": grp["condition_count"].sum(),
        "total_bed_days_open": grp["bed_days_open"].sum()
    }))
    .assign(
        conditions_per_10000_bed_days = 
        lambda df: df["conditions"] / df["total_bed_days_open"] * 10000)
)

In [23]:
overall_results

facs  surveys  conditions  total_bed_days_open  \
state is_brightspring                                                   
CA    False             868     3827        95.0             23358694   
      True               87      650        28.0              3944340   
IN    False             227      438       129.0              3158868   
      True               76      203        64.0              1567908   
LA    False             100      128        40.0               915192   
      True               36       58        20.0               423612   
NC    False             130      264        12.0              1623336   
      True               30       74         4.0               456144   
OH    False              33       46         8.0               352208   
      True               22       34        14.0               262040   
TX    False             509     1212        23.0              7453519   
      True              178      498        74.0              3058848   
WV    False               4        4         4.0                27864   
      True               17       28        43.0               206736   

                       conditions_per_10000_bed_days  
state is_brightspring                                 
CA    False                                 0.040670  
      True                                  0.070988  
IN    False                                 0.408374  
      True                                  0.408187  
LA    False                                 0.437067  
      True                                  0.472130  
NC    False                                 0.073922  
      True                                  0.087692  
OH    False                                 0.227139  
      True                                  0.534270  
TX    False                                 0.030858  
      True                                  0.241921  
WV    False                                 1.435544  
      True                                  2.079947

The final results below are BrightSpring's total conditions per 10,000 bed days for 4-8 bed facilities (**True** column) versus everyone else (**False** column), the ratio of the two values, and the difference of the two values for each state.

In [24]:
(
    overall_results["conditions_per_10000_bed_days"]
    .unstack()
    .assign(vs_rest_ratio = lambda df: df[True] / df[False])
    .assign(vs_rest_difference = lambda df: df[True] - df[False])
    .round(2)
    .sort_values("vs_rest_ratio", ascending=False)
)

is_brightspring,False,True,vs_rest_ratio,vs_rest_difference
state,,,,
TX,0.03,0.24,7.84,0.21
OH,0.23,0.53,2.35,0.31
CA,0.04,0.07,1.75,0.03
WV,1.44,2.08,1.45,0.64
NC,0.07,0.09,1.19,0.01
LA,0.44,0.47,1.08,0.04
IN,0.41,0.41,1.00,-0.00


----

----

----